## Exercise 3: Textual Data Analysis
##### Jade Watson: 20052115

### Data Preprocessing

In [4]:
import pandas as pd
import numpy as np
import os
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

datasetPath = r"C:/Users/Jade Watson/Documents/CMPE 351/Exercise3//"
print(os.listdir(datasetPath))

['.idea', 'CISC_CMPE351_W22_E3.pdf', 'corpus.pkl', 'dictionary.gensim', 'Exercise3.ipynb', 'JadeWatsonExercise3.ipynb', 'model1.gensim', 'model1.gensim.expElogbeta.npy', 'model1.gensim.id2word', 'model1.gensim.state', 'model10.gensim', 'model10.gensim.expElogbeta.npy', 'model10.gensim.id2word', 'model10.gensim.state', 'model5.gensim', 'model5.gensim.expElogbeta.npy', 'model5.gensim.id2word', 'model5.gensim.state', 'Questions.csv', 'Questions.csv.zip', 'run_word2vec.ipynb', 'w2v_model.txt']


In [5]:
# Style Dataframe
df = pd.read_csv(datasetPath + "Questions.csv")
df = df.sample(frac=1).reset_index(drop=True)
df.head(10)

,Id,OwnerUserId,CreationDate,Score,Title,Body
0,43430325,7872366.0,2017-04-15T19:25:31Z,3,How do I split a txt file by html tags or rege...,<p>I have the output of a LexisNexis batch dow...
1,44906264,2659466.0,2017-07-04T12:39:48Z,0,Add speaker notes to beamer presentations usin...,<p>I want to create a beamer pdf presentation ...
2,38765996,5392289.0,2016-08-04T11:21:53Z,2,Fit Weibull to distribution with genextreme an...,"<p>Using SciPy, I am trying to reproduce the w..."
3,42078862,6452857.0,2017-02-06T23:01:53Z,3,R: How to combine rows of a data frame with th...,<p>Example data frame</p>\n\n<pre><code>date ...
4,23961089,2743953.0,2014-05-30T18:23:14Z,0,"Flat-File Authentication, login form does not ...","<p>I am evaluating <a href=""http://rstudio.git..."
5,11252407,720300.0,2012-06-28T20:29:36Z,4,XPath to extract text after br tags in R,<p>How to extract text after the <code>br</cod...
6,21534365,1515626.0,2014-02-03T18:03:41Z,0,Recoding variable into an ordered factor varia...,<p>I am following the documentation from the <...
7,13376203,1522730.0,2012-11-14T09:35:33Z,1,Employ environments to handle package-data in ...,<p>I recently wrote a R extension. The functio...
8,45958533,7100507.0,2017-08-30T10:52:54Z,0,Plot individual-level glm-fit on aggregate-lev...,<p>In R I am fitting a GLM (logit) on individu...
9,33068008,5212690.0,2015-10-11T17:33:40Z,0,how to delete the first n characters of rownam...,<p>I have a data-frame which looks like this:<...


#### Cleaning
##### Lemmatizing and removing the stopwords and non-alphabetic characters for each line of dialogue.

In [6]:
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize

df['Title'] =df['Title'].str.replace("[^a-zA-Z#]", " ")
stopwords_list = stopwords.words('english')
punctuations = list(set(string.punctuation))

def clean_text_initial(text):
    text = ' '.join([x.lower() for x in word_tokenize(text) if x.lower() not in stopwords_list and len(x)>1])
    text = ' '.join([x.lower() for x in word_tokenize(text) if x.lower() not in punctuations and len(x)>3])
    return text.strip()

df["clean_text"]=df.Title.apply(lambda text:clean_text_initial(str(text)))
df.head()

[nltk_data] Downloading package stopwords to C:\Users\Jade
[nltk_data]     Watson\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Jade Watson\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
C:\Users\Jade Watson\AppData\Local\Temp\ipykernel_5684\2985933192.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Title'] =df['Title'].str.replace("[^a-zA-Z#]", " ")


,Id,OwnerUserId,CreationDate,Score,Title,Body,clean_text
0,43430325,7872366.0,2017-04-15T19:25:31Z,3,How do I split a txt file by html tags or rege...,<p>I have the output of a LexisNexis batch dow...,split file html tags regex order save separate...
1,44906264,2659466.0,2017-07-04T12:39:48Z,0,Add speaker notes to beamer presentations usin...,<p>I want to create a beamer pdf presentation ...,speaker notes beamer presentations using rmark...
2,38765996,5392289.0,2016-08-04T11:21:53Z,2,Fit Weibull to distribution with genextreme an...,"<p>Using SciPy, I am trying to reproduce the w...",weibull distribution genextreme weibull
3,42078862,6452857.0,2017-02-06T23:01:53Z,3,R How to combine rows of a data frame with th...,<p>Example data frame</p>\n\n<pre><code>date ...,combine rows data frame take newest value
4,23961089,2743953.0,2014-05-30T18:23:14Z,0,Flat File Authentication login form does not ...,"<p>I am evaluating <a href=""http://rstudio.git...",flat file authentication login form shown shin...


I decided to lemmanize the titles instaed of stemming them directly because it stems the word but ensures that it does not loose its meaning. Lemmatization has a pre-defined dictionary that stores the context of words and checks the word in the dictionary.

In [7]:
from gensim.corpora.dictionary import Dictionary

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

cleaned_text_list=df.clean_text.apply(lambda clean_text:[lemmatizer.lemmatize(tokenized_text) for tokenized_text in word_tokenize(clean_text)])

gensim_dict=Dictionary(cleaned_text_list)

doc_term_matrix = [gensim_dict.doc2bow(text) for text in cleaned_text_list]

LookupError: 
**********************************************************************
  Resource [93momw-1.4[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('omw-1.4')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/omw-1.4[0m

  Searched in:
    - 'C:\\Users\\Jade Watson/nltk_data'
    - 'C:\\Users\\Jade Watson\\anaconda3\\nltk_data'
    - 'C:\\Users\\Jade Watson\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\Jade Watson\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Jade Watson\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
print(df.clean_text)

### Bigrams
##### Used to detect common phrases from a list of sentances

In [ ]:
from gensim.models.phrases import Phrases, Phraser

# make list a series
clean_text_series = pd.Series(cleaned_text_list)

sent = [row.split() for row in df.clean_text]
# create the relevant phrases from the list of sentances
phrases = Phrases(sent,min_count=30,progress_per=10000)
# use phraser() to cut down memory consumption of phrases() by discarding model state not needed for bigram detection task
bigram = Phraser(phrases)
# transform the corpus based on the bigrams detected:
sentences = bigram[sent]

### Most Frequent Words
##### Used to check the effectiveness of the lemmatization and addition of bigrams

In [ ]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

In [ ]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

### Training the Model

##### i) Word2Vec(): set up parameters of the model


In [ ]:
import multiprocessing
from gensim.models import Word2Vec
# count the number of cores in a computer
cores = multiprocessing.cpu_count()

In [ ]:
w2v_model = Word2Vec(min_count=25,window=2,size=300,sample=6e-5,alpha=0.03,min_alpha=0.0007,negative=20,workers=cores-1)

##### ii) Build Vocabulary Table: build vocab from a sequence of sentences to initialize the model

In [ ]:
# build the vocabulary table
t = time()
w2v_model.build_vocab(sentences, progress_per=10000)
print('Time to build vocab table: {} mins'.format(round((time()-t)/60,2)))

##### iii) Train the model

In [ ]:
t = time()
# where total_examples is the count of sentences
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
# make model more memory-efficient
w2v_model.init_sims(replace=True)

In [ ]:
learned_words = list(w2v_model.wv.vocab)
print(learned_words)

In [ ]:
# save model
w2v_model.wv.save_word2vec_format('w2v_model.txt')

### Exploring the Model

In [ ]:
w2v_model.wv.most_similar(positive=['plots'])

In [ ]:
w2v_model.wv.similarity('plot','answer')

### Topic Modelling

In [ ]:
from gensim import corpora

dictionary = corpora.Dictionary(sentences)
corpus = [dictionary.doc2bow(text) for text in sentences]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [ ]:
import gensim

NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

In [ ]:
from gensim.models import CoherenceModel

# Compute Perplexity
print('\nPerplexity: ', ldamodel.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

In [ ]:
# 10 topics
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
ldamodel.save('model10.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', ldamodel.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

### Part 5 Summary

According to our notes, calculating the perplexity is how one chooses the number of topics. To calculate perplexity the inverse log-likelihood of the dictionary above is taken. We wish to have models with lower perplexity because it suggests less uncertainties about the unobserved document. I tested this theory by varying the number of topics. I first tested with 5 topics. It is evident that topic 1 contains words like function, using, loop, and error. Topics 1-4 are printed above as well. Next I tested with 10 topics. Topic 6 contains words like data, table, extract, and remove. The weights in front of each words reflects how important the keyword is to the topic. So the higher the weight the more it contributes to the topic. In the case above, the preferred number of topics is 5. Another approach to finding the optimal number of topics is to build many LDA models with different topic number and pick one that gives the highest coherence value. To do this, you can use the CoherenceModel(). I did not do this because the runtime to build each LDA model is long.